<a href="https://colab.research.google.com/github/FelixSchmidt92/question_generation/blob/master/question_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data preparation for training the answer aware question generation model.

In [43]:
!git clone https://github.com/FelixSchmidt92/question_generation.git
%cd question_generation

Cloning into 'question_generation'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 9 (delta 0), reused 9 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.
/content/gdrive/My Drive/question_generation


In [44]:
!python build_dataset.py

File downloaded successfully!
File downloaded successfully!
100% 442/442 [02:38<00:00,  3.40it/s]
100% 35/35 [00:17<00:00,  1.64it/s]


In [46]:
%cd ..
!git clone https://github.com/OpenNMT/OpenNMT-py.git
%cd OpenNMT-py
!python setup.py install

/content/gdrive/My Drive
Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15547 (delta 6), reused 6 (delta 3), pack-reused 15529
Receiving objects: 100% (15547/15547), 146.48 MiB | 20.77 MiB/s, done.
Resolving deltas: 100% (11192/11192), done.
Checking out files: 100% (197/197), done.
/content/gdrive/My Drive/OpenNMT-py
running install
running bdist_egg
running egg_info
creating OpenNMT_py.egg-info
writing OpenNMT_py.egg-info/PKG-INFO
writing dependency_links to OpenNMT_py.egg-info/dependency_links.txt
writing entry points to OpenNMT_py.egg-info/entry_points.txt
writing requirements to OpenNMT_py.egg-info/requires.txt
writing top-level names to OpenNMT_py.egg-info/top_level.txt
writing manifest file 'OpenNMT_py.egg-info/SOURCES.txt'
writing manifest file 'OpenNMT_py.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_l

In [47]:
!pip install torchtext==0.4.0

In [48]:
%cd ../question_generation/data/squad/

/content/gdrive/My Drive/question_generation/data/squad


In [0]:
mkdir ../preprocessed

In [50]:
!python preprocess.py -train_src ../question_generation/data/squad/train/train.sentence -train_tgt ../question_generation/data/squad/train/train.question -valid_src ../question_generation/data/squad/dev/dev.sentence -valid_tgt ../question_generation/data/squad/dev/dev.question -save_data ../preprocessed/data -src_vocab_size 45000 -tgt_vocab_size 28000 -src_seq_length 100

python3: can't open file 'preprocess.py': [Errno 2] No such file or directory


In [36]:
%cd /preprocessed/
!mkdir glove
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d glove

/content/preprocessed
--2020-03-07 14:28:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-03-07 14:28:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-03-07 14:28:34--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘

In [39]:
%cd ../OpenNMT-py/
!python ./tools/embeddings_to_torch.py -emb_file_both ../preprocessed/glove/glove.6B.300d.txt \
-dict_file ../preprocessed/data.vocab.pt \
-output_file ../preprocessed/embeddings


/content/OpenNMT-py
[2020-03-07 14:42:54,060 INFO] From: ../preprocessed/data.vocab.pt
[2020-03-07 14:42:54,061 INFO] 	* source vocab: 45002 words
[2020-03-07 14:42:54,061 INFO] 	* target vocab: 28004 words
[2020-03-07 14:42:54,073 INFO] Reading encoder and decoder embeddings from ../preprocessed/glove/glove.6B.300d.txt
[2020-03-07 14:43:02,623 INFO] 	Found 400000 total vectors in file
[2020-03-07 14:43:02,623 INFO] After filtering to vectors in vocab:
[2020-03-07 14:43:02,646 INFO] 	* enc: 23747 match, 21255 missing, (52.77%)
[2020-03-07 14:43:02,659 INFO] 	* dec: 14376 match, 13628 missing, (51.34%)
[2020-03-07 14:43:02,659 INFO] 
Saving embedding as:
	* enc: ../preprocessed/embeddings.enc.pt
	* dec: ../preprocessed/embeddings.dec.pt
[2020-03-07 14:43:03,587 INFO] 
Done.


Save files do drive to continue training elsewhere.

In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd ../gdrive/My\ Drive
%mkdir question_generation_preprocessed
%cp -r ../../preprocessed/data.train.0.pt question_generation_preprocessed
%cp -r ../../preprocessed/data.valid.0.pt question_generation_preprocessed
%cp -r ../../preprocessed/embeddings.dec.pt question_generation_preprocessed
%cp -r ../../preprocessed/embeddings.enc.pt question_generation_preprocessed
%cp -r ../../preprocessed/data.vocab.pt question_generation_preprocessed

The following command starts the training. But it should be run somewhere else (not colab) because the runtime session will terminate after some hours...

In [59]:
%cd /content/
!mkdir save_dir
%cd OpenNMT-py/

/content
/content/OpenNMT-py


In [0]:
!python train.py -save_model ../save_dir -gpu_ranks 0 -rnn_type LSTM -encoder_type brnn -world_size 1 -enc_layers 2 -dec_layers 2 -dropout 0.3 -optim adam -learning_rate 1 -layers 2 -bridge -global_attention general -pre_word_vecs_enc "../preprocessed/embeddings.enc.pt" -pre_word_vecs_dec "../preprocessed/embeddings.dec.pt" -rnn_size 600 -word_vec_size 300 -fix_word_vecs_enc -fix_word_vecs_dec -data ../preprocessed/data
